In [33]:
import soundfile as sf
from IPython.display import Audio
import numpy as np
import random 
import pandas as pd
import os
from os.path import join as pjoin
import tqdm
from acoustics.bands import third
import scipy.signal as sig
from IPython.display import Audio
# from masp import shoebox_room_sim as srs
from scipy.io import wavfile
#import mat73
import matplotlib.pyplot as plt

In [34]:
# import my modules:
import sys
sys.path.append('../src')
import joa_helpers as hlp
import importlib
importlib.reload(hlp);

In [35]:
np.random.seed(42)
pd.np.random.seed(42) 

AttributeError: module 'pandas' has no attribute 'np'

In [36]:
# ---------- NOISE POOL ------------
# create df with paths to noise files 
noise_path='/home/ubuntu/Data/wham'
database="wham"
noise_pool = []
for split in ['tr', 'cv', 'tt']:
    split_path = pjoin(pjoin(noise_path, 'wham_noise'), split)
    for audio_path in os.listdir(split_path):
        noise_pool.append({'database_noise': database, 'noise_file_path': pjoin(split_path,audio_path)})

noise_pool=pd.DataFrame(noise_pool)
print(f"{len(noise_pool)=}")

noise_pool.to_csv("noise_pool.csv")
noise_pool.head(10)

len(noise_pool)=28000


,database_noise,noise_file_path
0,wham,/home/ubuntu/Data/wham/wham_noise/tr/012o030z_...
1,wham,/home/ubuntu/Data/wham/wham_noise/tr/01ra010k_...
2,wham,/home/ubuntu/Data/wham/wham_noise/tr/01zc0206_...
3,wham,/home/ubuntu/Data/wham/wham_noise/tr/20pa010i_...
4,wham,/home/ubuntu/Data/wham/wham_noise/tr/208a0105_...
5,wham,/home/ubuntu/Data/wham/wham_noise/tr/020c020l_...
6,wham,/home/ubuntu/Data/wham/wham_noise/tr/40ca010k_...
7,wham,/home/ubuntu/Data/wham/wham_noise/tr/408c020o_...
8,wham,/home/ubuntu/Data/wham/wham_noise/tr/01cc0211_...
9,wham,/home/ubuntu/Data/wham/wham_noise/tr/20eo010h_...


In [37]:
# ---------- SPEECH POOL ------------

# create df with paths to speech files
speech_dataset_path1 = '/home/ubuntu/Data/VCTK/wav48_silence_trimmed/'
speech_dataset_path2 = '/home/ubuntu/Data/LibriMix/LibriSpeech'
speech_dataset_path3 = '/home/ubuntu/Data/PTDB'

# initialize empty list of files:
speech_pool = []

# fill the list of files with filenames from vctk data base:
database="VCTK"
for root, dirs, files in os.walk(speech_dataset_path1):
    for file in files:
        if file.endswith('.flac'):
            # decide which split based on a probability 
            speech_pool.append({'database_speech': database, 'speech_file_path': os.path.join(root, file)})

# fill the list of files with filenames from librispeech data base:
database="librispeech"
for split in ['train-clean-100', 'test-clean', 'dev-clean']:
    split_path = pjoin(pjoin(speech_dataset_path2, split))
    for root, dirs, files in os.walk(split_path):
        for file in files:
            if file.endswith('.flac'):
                speech_pool.append({'database_speech': database, 'speech_file_path': os.path.join(root, file)})

# fill the list of files with filenames from ptdb data base:
database="PTDB"
for root, dirs, files in os.walk(speech_dataset_path3):
    for file in files:
        # make sure the correct speech files are used (MIC directory)
        if ("/MIC/" in root) & (file.endswith('.wav')):
            # decide which split based on a probability 
            speech_pool.append({'database_speech': database, 'speech_file_path': os.path.join(root, file)})  

# shuffle order
random.shuffle(speech_pool)

# turn list to data frame 
speech_pool = pd.DataFrame(speech_pool)
print(f"{len(speech_pool)=}")

speech_pool.to_csv("speech_pool.csv")
speech_pool.head(10)

len(speech_pool)=126908


,database_speech,speech_file_path
0,VCTK,/home/ubuntu/Data/VCTK/wav48_silence_trimmed/p...
1,VCTK,/home/ubuntu/Data/VCTK/wav48_silence_trimmed/p...
2,VCTK,/home/ubuntu/Data/VCTK/wav48_silence_trimmed/p...
3,librispeech,/home/ubuntu/Data/LibriMix/LibriSpeech/train-c...
4,librispeech,/home/ubuntu/Data/LibriMix/LibriSpeech/train-c...
5,VCTK,/home/ubuntu/Data/VCTK/wav48_silence_trimmed/p...
6,VCTK,/home/ubuntu/Data/VCTK/wav48_silence_trimmed/p...
7,VCTK,/home/ubuntu/Data/VCTK/wav48_silence_trimmed/p...
8,VCTK,/home/ubuntu/Data/VCTK/wav48_silence_trimmed/p...
9,VCTK,/home/ubuntu/Data/VCTK/wav48_silence_trimmed/s...


In [38]:
# ---------- RIR POOL ------------
rir_path='/home/ubuntu/Data/synth_rirs_mono'
database="synth_rirs_mono"
# load df with rirs paths and stats:
rir_pool=pd.read_csv(pjoin(rir_path,"rir_info.csv"),index_col=0)
rir_pool["database_rir"]=database
print(f"{len(rir_pool)=}")
rir_pool.to_csv("rir_pool.csv")
rir_pool.head(10)

len(rir_pool)=10000


,room_x,room_y,room_z,volume,rt60_set,mic_pos_x,mic_pos_y,mic_pos_z,src_pos_x,src_pos_y,...,rt60_true,rt60_masp_stats,cd_masp_stats,mfp_masp_stats,rt30_meas,rt20_meas,edt_meas,c50_meas,ir_file_path,database_rir
0,3.087404,1.733557,2.544188,13.616981,0.094051,1.275112,0.636568,1.717796,1.375112,0.636568,...,0.094054,0.094054,0.678853,1.545836,0.087845,0.090524,0.222644,29.869119,/home/ubuntu/Data/synth_rirs_mono/monoRIR_x309...,synth_rirs_mono
1,3.120417,1.627469,2.705770,13.740938,0.098842,1.533229,0.775045,1.112430,1.633229,0.775045,...,0.098846,0.098838,0.665230,1.533153,0.108747,0.107866,0.212617,25.080770,/home/ubuntu/Data/synth_rirs_mono/monoRIR_x312...,synth_rirs_mono
2,3.061378,1.560564,3.018367,14.420179,0.096717,1.878979,0.702089,1.150549,1.978979,0.702089,...,0.096723,0.096712,0.688923,1.539943,0.096810,0.093025,0.235392,26.606241,/home/ubuntu/Data/synth_rirs_mono/monoRIR_x306...,synth_rirs_mono
3,3.076184,1.544429,3.316970,15.758751,0.094286,1.419961,0.712446,1.134668,1.519961,0.712446,...,0.094291,0.094282,0.729411,1.569801,0.087765,0.089848,0.230418,27.731729,/home/ubuntu/Data/synth_rirs_mono/monoRIR_x308...,synth_rirs_mono
4,3.015096,1.828398,3.086247,17.013851,0.100507,1.599783,0.966029,1.815395,1.699783,0.966029,...,0.100508,0.100506,0.734059,1.663050,0.099996,0.100491,0.220712,28.897935,/home/ubuntu/Data/synth_rirs_mono/monoRIR_x302...,synth_rirs_mono
5,3.079932,1.645339,3.434983,17.406880,0.107771,1.097511,0.717791,1.034888,1.197511,0.717791,...,0.107776,0.107776,0.717009,1.634545,0.106246,0.109946,0.233664,22.603079,/home/ubuntu/Data/synth_rirs_mono/monoRIR_x308...,synth_rirs_mono
6,3.478583,1.810856,2.799080,17.632004,0.096148,2.047192,0.964826,1.030445,2.147192,0.964826,...,0.096153,0.096144,0.764039,1.670902,0.093377,0.096400,0.196960,28.776429,/home/ubuntu/Data/synth_rirs_mono/monoRIR_x348...,synth_rirs_mono
7,3.222587,1.953429,2.803276,17.646888,0.103874,1.995593,1.066677,1.723879,2.095593,1.066677,...,0.103878,0.103869,0.735387,1.696417,0.099488,0.097691,0.225181,26.747138,/home/ubuntu/Data/synth_rirs_mono/monoRIR_x322...,synth_rirs_mono
8,3.094981,1.597018,3.713484,18.354788,0.092483,1.948308,0.915701,1.922755,2.048308,0.915701,...,0.092484,0.092484,0.794817,1.641282,0.088072,0.092394,0.212735,28.558985,/home/ubuntu/Data/synth_rirs_mono/monoRIR_x309...,synth_rirs_mono
9,3.136293,1.779292,3.310702,18.474978,0.093152,1.900544,0.934894,1.239519,2.000544,0.934894,...,0.093153,0.093153,0.794544,1.690732,0.082575,0.086574,0.208506,32.160277,/home/ubuntu/Data/synth_rirs_mono/monoRIR_x314...,synth_rirs_mono


In [39]:
# --------- CREATE A DATASET BEING COMBINATIONS OF SPEECH, NOISE AND RIRS --------

# create dataset with 20000 data points, which consists of random combinations of speech, noise and rirs
N_datapoints=50000

# sample from noise pool: 
df_noise=noise_pool.sample(N_datapoints,replace=True)
# here plan ways to augment noise data set:
random_bool_values = [random.choice([1, -1]) for _ in range(len(df_noise))]
df_noise["aug_phase"]=random_bool_values

# sample from speech pool: 
df_speech=speech_pool.sample(N_datapoints)

# sample from rir pool: 
df_rir=rir_pool.sample(N_datapoints,replace=True)

# concatenate samples from speech, noise and rir pools
df_ds = pd.concat([df_speech.reset_index(drop=True), df_noise.reset_index(drop=True), df_rir.reset_index(drop=True)], axis=1,ignore_index=False)
df_ds = df_ds.reset_index(drop=True)

# randomize snr (only high snrs)
df_ds["snr"]= np.random.uniform(low=10, high=30, size=len(df_ds))

# Create test-train-val split:
df_ds.loc[0:N_datapoints*0.8,"split"]="train" # 80% training data
df_ds.loc[N_datapoints*0.8:N_datapoints*0.9,"split"]="test" # 10% testing data
df_ds.loc[N_datapoints*0.9:N_datapoints,"split"]="val" # 10% validation data

df_ds.to_csv("data_set.csv")

In [40]:
df_ds.head(10)

,database_speech,speech_file_path,database_noise,noise_file_path,aug_phase,room_x,room_y,room_z,volume,rt60_set,...,cd_masp_stats,mfp_masp_stats,rt30_meas,rt20_meas,edt_meas,c50_meas,ir_file_path,database_rir,snr,split
0,VCTK,/home/ubuntu/Data/VCTK/wav48_silence_trimmed/p...,wham,/home/ubuntu/Data/wham/wham_noise/cv/01jc020n_...,-1,11.248648,7.560777,4.355760,370.450903,0.392010,...,1.734306,4.437126,0.434046,0.421117,0.395259,9.347300,/home/ubuntu/Data/synth_rirs_mono/monoRIR_x112...,synth_rirs_mono,21.377927,train
1,VCTK,/home/ubuntu/Data/VCTK/wav48_silence_trimmed/p...,wham,/home/ubuntu/Data/wham/wham_noise/tr/20do0103_...,-1,24.788887,13.210832,3.256179,1066.339506,0.708884,...,2.188171,4.726518,1.414155,1.454975,0.899070,5.494828,/home/ubuntu/Data/synth_rirs_mono/monoRIR_x247...,synth_rirs_mono,11.787477,train
2,VCTK,/home/ubuntu/Data/VCTK/wav48_silence_trimmed/p...,wham,/home/ubuntu/Data/wham/wham_noise/tr/20nc010j_...,-1,5.132828,2.814441,3.644390,52.647009,0.155430,...,1.038327,2.425627,0.157071,0.161133,0.241845,19.908559,/home/ubuntu/Data/synth_rirs_mono/monoRIR_x513...,synth_rirs_mono,11.368276,train
3,PTDB,/home/ubuntu/Data/PTDB/MALE/MIC/M01/mic_M01_sa...,wham,/home/ubuntu/Data/wham/wham_noise/tr/028o0311_...,-1,20.675004,16.943232,4.057225,1421.251637,0.764330,...,2.432863,5.651920,1.347417,1.301293,0.696367,6.049499,/home/ubuntu/Data/synth_rirs_mono/monoRIR_x206...,synth_rirs_mono,29.729116,train
4,VCTK,/home/ubuntu/Data/VCTK/wav48_silence_trimmed/p...,wham,/home/ubuntu/Data/wham/wham_noise/cv/20to0102_...,1,12.159545,11.248934,4.551145,622.514350,0.503858,...,1.983151,5.116893,0.612778,0.575082,0.411693,8.774874,/home/ubuntu/Data/synth_rirs_mono/monoRIR_x121...,synth_rirs_mono,26.379893,train
5,VCTK,/home/ubuntu/Data/VCTK/wav48_silence_trimmed/p...,wham,/home/ubuntu/Data/wham/wham_noise/tr/40jc0218_...,-1,13.730542,9.092060,2.847435,355.470693,0.414196,...,1.652741,3.745250,0.676085,0.647260,0.508989,7.275837,/home/ubuntu/Data/synth_rirs_mono/monoRIR_x137...,synth_rirs_mono,17.626652,train
6,VCTK,/home/ubuntu/Data/VCTK/wav48_silence_trimmed/p...,wham,/home/ubuntu/Data/wham/wham_noise/tr/01io030x_...,1,29.326935,20.979795,3.370734,2073.921931,0.949096,...,2.637446,5.284930,2.123580,2.165348,1.187067,5.980619,/home/ubuntu/Data/synth_rirs_mono/monoRIR_x293...,synth_rirs_mono,25.252472,train
7,librispeech,/home/ubuntu/Data/LibriMix/LibriSpeech/train-c...,wham,/home/ubuntu/Data/wham/wham_noise/cv/015c020c_...,-1,9.637075,8.158496,4.502014,353.966511,0.353122,...,1.786247,4.459703,0.364523,0.358536,0.296502,11.760081,/home/ubuntu/Data/synth_rirs_mono/monoRIR_x964...,synth_rirs_mono,10.000525,train
8,VCTK,/home/ubuntu/Data/VCTK/wav48_silence_trimmed/p...,wham,/home/ubuntu/Data/wham/wham_noise/tr/028o030j_...,1,3.931324,2.748257,3.524458,38.079254,0.129586,...,0.967135,2.217379,0.106379,0.109448,0.250142,26.140992,/home/ubuntu/Data/synth_rirs_mono/monoRIR_x393...,synth_rirs_mono,19.948927,train
9,VCTK,/home/ubuntu/Data/VCTK/wav48_silence_trimmed/p...,wham,/home/ubuntu/Data/wham/wham_noise/tr/022o030k_...,1,7.033984,6.734177,3.905954,185.017603,0.291363,...,1.421691,3.658430,0.293000,0.293387,0.376620,11.925949,/home/ubuntu/Data/synth_rirs_mono/monoRIR_x703...,synth_rirs_mono,10.012854,train
